In [41]:
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

**Load MNIST and Preprocess the dataset**

In [42]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# cast the data to floats in order to work with decimal division
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Normalize the vectors codes
# Initially, their maximum value is 255
x_train = x_train / 255
x_test = x_test / 255

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


**Building the convolutional network using Tenserflow**

In [43]:
model = Sequential()

# Convolution layer with L2 regularization = Feature extractor
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu",
                 kernel_regularizer=l2(1e-4),
                 input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten + hidden layer = Classifier
model.add(Flatten())
model.add(Dense(128, activation="relu", kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.3)) # Dropout to reduce overfitting
model.add(Dense(10, activation="softmax"))

# Early stopping
early_stopping_monitor = EarlyStopping(
    monitor="val_loss",
    patience=1,
    mode="min", # "min" for val_loss, "max" for val_accuracy
    restore_best_weights=True,
)

print("\nCompile and fit the model")

# Compile
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Fit
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,     # bigger batch size = faster on GPU
    epochs=20,          # but will stop early if no improvement
    validation_split=0.2,
    callbacks=[early_stopping_monitor],
    verbose=1,
)

# Evaluate
print("\nEvaluate the model")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"✅ Test accuracy: {test_acc:.4f}")

# Save
model.save("trained_model.keras")



Compile and fit the model
Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8334 - loss: 0.5877 - val_accuracy: 0.9722 - val_loss: 0.1360
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9638 - loss: 0.1600 - val_accuracy: 0.9794 - val_loss: 0.1128
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9747 - loss: 0.1234 - val_accuracy: 0.9827 - val_loss: 0.1029
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9790 - loss: 0.1100 - val_accuracy: 0.9830 - val_loss: 0.0976
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9826 - loss: 0.0986 - val_accuracy: 0.9852 - val_loss: 0.0909
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9856 - loss: 0.0886 - val_accuracy: 0.9858 - val_loss: 0.0899
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9862 - loss: 0.0855 - val_accuracy: 0.9852 - val_loss: 0.0906

Evaluate the model
✅ Test accuracy: 0.9851
